# Импорты

In [1]:
!pip install tabula-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 4.8 MB/s 


In [2]:
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.3 MB/s 


In [3]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.1 MB/s 


In [4]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 1.5 MB 9.6 MB/s 
     |████████████████████████████████| 42 kB 999 kB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 65 kB 2.5 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=24df3cc7003b8082a0a4b23a7b5d2408abfbb81d216c92619e277b2072a13fc2
  Stored in directory: /root/.cache/pip/wheels/dd/59/af/8d6c96a719763990f1c548e36b17d9efdfb767f42f7ff39f53
Successfully built googletrans


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import VotingClassifier, VotingRegressor
from sklearn.ensemble import StackingClassifier,StackingRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.model_selection import GridSearchCV,KFold,train_test_split,cross_val_score,StratifiedKFold
from sklearn.metrics import mean_squared_error,accuracy_score,precision_score
from sklearn.metrics import balanced_accuracy_score, top_k_accuracy_score, average_precision_score, brier_score_loss
from sklearn.metrics import f1_score,  log_loss, precision_score, recall_score, jaccard_score, roc_auc_score,classification_report
import catboost as cb
from joblib import dump, load
from tqdm.notebook import trange, tqdm
from itertools import combinations_with_replacement, combinations, product
import warnings
import math
import joblib
import requests
from urllib import request
from urllib.error import HTTPError, URLError
import tabula
import unidecode
from bs4 import BeautifulSoup

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Исследование
В этой части проекта вы должны подготовить все, что используется в классах и методах, описанных выше, в Jupyter Notebook (recipes.ipynb).

## 1. Прогноз

### 1. Подготовка данных

* Используйте датасет из Epicurious, подготовленный HugoDarwood
* Отфильтруйте поля: удалите как можно больше столбцов, которые не относятся к названиями ингридиентов. Вы будете прогнозировать оценку или категорию оценки, используя только ингридиенты и ничего больше. Если модель обучится на других данных, которые не будут передаваться на вход, то точность прогнозов будет заведомо ниже.

In [7]:
path = "/content/drive/My Drive/school21/ds-project-15/task/datasets"
%cd $path

/content/drive/My Drive/school21/ds-project-15/task/datasets


In [154]:
df = pd.read_csv('epi_r.csv')
df

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,28.0,2.0,2.0,64.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,671.0,22.0,28.0,583.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,563.0,31.0,38.0,652.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,631.0,45.0,24.0,517.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ingridients = pd.DataFrame(columns=['en'], data=list(df.columns.values))

In [ ]:
import googletrans
from googletrans import Translator
translator = Translator()

In [ ]:
for index, row in ingridients.iterrows():
  ingridients.loc[index, 'ru'] = translator.translate(row.en, src='en', dest='ru').text

In [ ]:
ingridients

,en,ru
0,title,заглавие
1,rating,рейтинг
2,calories,калории
3,protein,белок
4,fat,толстый
...,...,...
675,cookbooks,поваренные книги
676,leftovers,остатки
677,snack,перекус
678,snack week,неделя закусок


In [ ]:
ingridients.to_csv('ingridients.csv',index=False)

*Отмечаем вручную в выгруженном файле возможные ингридиенты отметками "yes"/"no"*

In [155]:
is_ingridients = pd.read_csv('is_ingridients.csv')
is_ingridients

,en,ru,ing
0,title,заглавие,yes
1,rating,рейтинг,yes
2,calories,калории,no
3,protein,белок,no
4,fat,толстый,no
...,...,...,...
675,cookbooks,поваренные книги,no
676,leftovers,остатки,no
677,snack,перекус,no
678,snack week,неделя закусок,no


In [156]:
for i in range(len(list(is_ingridients['en']))):
    if is_ingridients.iloc[i,2] == 'no':
        del df[is_ingridients.iloc[i,0]]

In [157]:
df

,title,rating,alcoholic,almond,amaretto,anchovy,anise,apple,apple juice,apricot,...,watermelon,wheat/gluten-free,whiskey,white wine,wild rice,wine,yogurt,yuca,zucchini,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
df.to_csv('epi_r_new.csv',index = False)

In [ ]:
df = pd.read_csv('epi_r_new.csv')

In [160]:
df_new = df.copy()
df_new.drop_duplicates(inplace=True)
y = df_new['rating']
del df_new['title'] 
del df_new['rating'] 
X = df_new

In [ ]:
scaler = StandardScaler()
feature_col_list = X.columns.tolist()
X[feature_col_list] = scaler.fit_transform(X[feature_col_list])
X

,alcoholic,almond,amaretto,anchovy,anise,apple,apple juice,apricot,artichoke,arugula,...,watermelon,wheat/gluten-free,whiskey,white wine,wild rice,wine,yogurt,yuca,zucchini,turkey
0,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,5.223962,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,7.094670
1,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
2,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
3,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
4,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20046,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
20047,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
20048,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,10.708785,-0.125041,...,-0.064974,1.719090,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
20050,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

### 2. Регрессия

* Для прогнозирования рейтинга опробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе gridsearch и кроссвалидации и оцените RMSE на тестовой подвыборке.

### LinearRegression

In [ ]:
params = {'fit_intercept': (True, False)}
lr = GridSearchCV(LinearRegression(), params, scoring='neg_mean_squared_error', n_jobs=-1, verbose=3)
lr.fit(X_train, y_train)
lr.best_params_

Fitting 5 folds for each of 2 candidates, totalling 10 fits


{'fit_intercept': True}

In [ ]:
lr_best=LinearRegression(fit_intercept=True)
print('RMSE LinearRegression :',cross_val_score(lr_best, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

RMSE LinearRegression : -3.1040156885242005e+25


### SVC

In [ ]:
params = {'kernel': ['linear', 'rbf', 'sigmoid'],
                  'C': (0.01, 0.1, 1, 1.5, 5, 10),
                  'gamma': ['scale', 'auto'],
                  'class_weight': ['balanced', None]}
svc = GridSearchCV(SVC(random_state=21), param_grid=params)
svc.fit(X_train, y_train)
svc.best_params_                

In [ ]:
svc_best=DecisionTreeRegressor(max_depth=5, max_features=200, min_samples_split=100)
print('RMSE SVC :',cross_val_score(svc_best, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

### DecisionTree

In [ ]:
params = {'max_depth' : [5,10,50,100,200],
              'min_samples_split' : [5,10,50,100,200],
              'max_features' : [5,10,50,100,200]}
dtr = GridSearchCV(DecisionTreeRegressor(), params, n_jobs=-1, return_train_score = True)
dtr.fit(X_train, y_train)
dtr.best_params_

{'max_depth': 5, 'max_features': 200, 'min_samples_split': 200}

In [ ]:
dtr_best=DecisionTreeRegressor(max_depth=5, max_features=200, min_samples_split=200)
print('RMSE DecisionTree :',cross_val_score(dtr_best, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

RMSE DecisionTree : -1.6041335501082028


### RandomForest

In [ ]:
params_rfr = {'n_estimators' : [10,50,100],
                 'max_depth' : [10,50,100],
                 'max_features' : [10,50,100]
                 }
rfr = GridSearchCV(RandomForestRegressor(random_state=21), params_rfr, cv=10, n_jobs=-1)
rfr.fit(X_train, y_train)
rfr.best_params_

{'max_depth': 10, 'max_features': 50, 'n_estimators': 50}

In [ ]:
rfr_best = RandomForestRegressor(max_depth= 50, max_features= 10, n_estimators= 50)
print('RMSE RandomForest :',cross_val_score(rfr_best, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

RMSE RandomForest : -1.5980941060805018


* Попробуйте различные ансамбли и их гиперпараметры. * Выберите лучшее решение на основе gridsearch и кроссвалидации и оцените RMSE на тестовой подвыборке.

### Bagging

In [ ]:
best_score = 0
i = 0
with tqdm(total=(5)) as pbar:
    for n_estimators in (5, 10, 25, 50, 100):
        br = BaggingRegressor(base_estimator= lr_best,
                              random_state=21, n_jobs=8, n_estimators=n_estimators)
        br.fit(X_train, y_train)
        br_score = cross_val_score(br, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

        if i == 0:
            best_score = br_score
            params = [n_estimators]
        
        if br_score > best_score:
            best_score = br_score
            params = [n_estimators]
        
        i =+ 1

        pbar.update(1)
print ("Лучший n_estimators =",params[0])
print ("Лучший Bagging RMSE для LinearRegression =",best_score)

  0%|          | 0/5 [00:00<?, ?it/s]

Лучший n_estimators = 50
Лучший Bagging RMSE для LinearRegression = -2.494207959870792e+24


In [ ]:
best2_score = 0
i = 0
with tqdm(total=(5)) as pbar:
    for n_estimators in (5, 10, 25, 50, 100):
        br2 = BaggingRegressor(base_estimator= dtr_best,
                              random_state=21, n_jobs=8, n_estimators=n_estimators)
        br2.fit(X_train, y_train)
        br2_score = cross_val_score(br2, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

        if i == 0:
            best2_score = br2_score
            params = [n_estimators]
        
        if br2_score > best2_score:
            best2_score = br2_score
            params = [n_estimators]
        
        i =+ 1

        pbar.update(1)
print ("Лучший n_estimators =",params[0])
print ("Лучший Bagging RMSE для DecisionTreeRegressor =",best2_score)

  0%|          | 0/5 [00:00<?, ?it/s]

Лучший n_estimators = 50
Лучший Bagging RMSE для DecisionTreeRegressor = -1.5887749933489823


In [ ]:
warnings.filterwarnings("ignore")
best3_score = 0
i = 0
with tqdm(total=(3)) as pbar:
    for n_estimators in (25, 50, 100):
        br3 = BaggingRegressor(base_estimator= rfr_best,
                              random_state=21, n_jobs=8, n_estimators=n_estimators)
        br3.fit(X_train, y_train)
        br3_score = cross_val_score(br3, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

        if i == 0:
            best3_score = br3_score
            params = [n_estimators]
        
        if br3_score > best3_score:
            best3_score = br3_score
            params = [n_estimators]
        
        i =+ 1

        pbar.update(1)
print ("Лучший n_estimators =",params[0])
print ("Лучший Bagging RMSE для RandomForestRegressor =",best3_score)

  0%|          | 0/3 [00:00<?, ?it/s]

Лучший n_estimators = 50
Лучший Bagging RMSE для RandomForestRegressor = -1.5747526856578797


### Stacking

In [ ]:
passthrough_par = []
n_splits_par = []
best_stack_score=0
i =0

for n in tqdm([2, 3, 4, 5, 6, 7]):
    k_fold = StratifiedKFold(n_splits=n, shuffle=True, random_state=21)
    stacking = StackingRegressor([('lr', lr_best),('dtr', dtr_best),('rfr', rfr_best)], passthrough = True,
                      final_estimator=RandomForestRegressor(),cv=7, n_jobs=-1)
    stacking_score = cross_val_score(stacking, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()
    if i ==0:
        passthrough_par = 'True'
        n_splits_par = n
        best_stack_score = stacking_score
    if stacking_score > best_stack_score:
        passthrough_par = 'True'
        n_splits_par = n
        best_stack_score = stacking_score        
    i=+1

for n in tqdm([2, 3, 4, 5, 6, 7]):

    k_fold = StratifiedKFold(n_splits=n, shuffle=True, random_state=21)
    stacking = StackingRegressor([('lr', lr_best),('dtr', dtr_best),('rfr', rfr_best)], passthrough = True,
                      final_estimator=RandomForestRegressor(),cv=7, n_jobs=-1)
    stacking_score = cross_val_score(stacking, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()  
    if stacking_score > best_stack_score:
        passthrough_par = 'False'
        n_splits_par = n
        best_stack_score = stacking_score     

print('Лучший passthrough = ', passthrough_par)
print('Лучший n_splits = ', n_splits_par)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Лучший passthrough =  True
Лучший n_splits =  7


In [ ]:
stacking = StackingRegressor([('lr', lr_best),('dtr', dtr_best),('rfr', rfr_best)], passthrough = False,
                      final_estimator=RandomForestRegressor(),cv=7, n_jobs=-1)
cross_val_score(stacking, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-1.769632462840736

Лучший ансамбль: Bagging на RandomForest

In [ ]:
bagging = BaggingRegressor(base_estimator= rfr_best, random_state=21, n_jobs=8, n_estimators=n_estimators).fit(X_train, y_train)
print("Лучший Bagging для RandomForestRegressor =",cross_val_score(br3, X_test, y_test, cv=10, scoring='neg_mean_squared_error').mean())

Лучший Bagging для RandomForestRegressor = -1.6167724635774554


* Посчитайте RMSE для наивного регрессора, в котором вы просто высчитали средний рейтинг.

In [ ]:
y_mean = y_test.mean()
y_pred = [y_mean for i in range(len(y_test))]
print("RMSE для наивного регрессора: ", math.sqrt(mean_squared_error(y_test, y_pred)))

RMSE для наивного регрессора:  1.3094201876736664


### 3. Классификация

* Бинаризируйте значения целевой переменной путем округления рейтингов до ближайшего целого числа. Это и будут ваши классы.

In [ ]:
target = round(y,0).astype(int)

In [ ]:
X_t_train, X_t_test, y_t_train, y_t_test = train_test_split(X, target, test_size=0.2, random_state=21, stratify=target)

* Для прогнозирования классов попробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе gridsearch и кроссвалидации и посчитайте accuracy на тестовой подвыборке.

Логистическая регрессия

In [ ]:
lr_c_parametrs = { 'penalty':('l1', 'l2'),
                   'fit_intercept':(True,False),
                   'solver':('lbfgs', 'liblinear', 'saga')}
lr_c_ = LogisticRegression(random_state=21)
lr_c = GridSearchCV(lr_c_, lr_c_parametrs, scoring='accuracy', n_jobs=-1)
lr_c.fit(X_t_train, y_t_train)
lr_c.best_params_

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py", line 447, in _check_solver
    raise ValueError(
ValueError: Solver l

{'fit_intercept': True, 'penalty': 'l2', 'solver': 'liblinear'}

In [ ]:
lr_c_best=LogisticRegression(random_state=21, fit_intercept = True, penalty = 'l2', solver = 'liblinear')
print("accuracy для LogisticRegression: ", cross_val_score(lr_c_best, X_t_train, y_t_train).mean())

accuracy для LogisticRegression:  0.6718975790531287


Дерево решений

In [ ]:
dtr_c_params = {'criterion': ('entropy', 'gini'),
                'class_weight': ['balanced', None],
                'max_depth': range(1, 50)}
dtr_c_ = DecisionTreeClassifier(random_state=21)
dtr_c = GridSearchCV(dtr_c_ , dtr_c_params, scoring='accuracy', n_jobs=-1)
dtr_c.fit(X_t_train, y_t_train)
dtr_c.best_params_

{'class_weight': None, 'criterion': 'gini', 'max_depth': 4}

In [ ]:
dtr_c_best=DecisionTreeClassifier(random_state=21,class_weight= None, criterion= 'gini', max_depth= 4)
print("accuracy для DecisionTree: ", cross_val_score(dtr_c_best, X_t_train, y_t_train).mean())

accuracy для DecisionTree:  0.6766340165942221


Случайный лес

In [ ]:
rfr_c_params = {'max_depth': tuple(range(1, 50)),
                'n_estimators': (10, 50, 100),
                'criterion': ('entropy', 'gini'),
                'class_weight': ['balanced', None]}
rfc_c_ = RandomForestClassifier(random_state=21)
rfc_c = GridSearchCV(rfc_c_ , rfr_c_params,  scoring='accuracy', n_jobs=-1)
rfc_c.fit(X_t_train, y_t_train)
rfc_c.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 17,
 'n_estimators': 50}

In [ ]:
rfc_c_best=RandomForestClassifier(random_state=21,class_weight=None,criterion='gini',max_depth= 17,n_estimators=50)
print("accuracy для RandomForestClassifier: ", cross_val_score(rfc_c_best, X_t_train, y_t_train).mean())

accuracy для RandomForestClassifier:  0.6741174444164214


In [ ]:
rfc_c_best=RandomForestClassifier(random_state=21,class_weight=None,criterion='gini',max_depth= 17,n_estimators=50)
print("accuracy для RandomForestClassifier: ", cross_val_score(rfc_c_best, X_t_test, y_t_test).mean())

accuracy для RandomForestClassifier:  0.6699228577690116


Наивный классификатор

* Рассчитайте значение accuracy наивного классификатора, в котором вы для всех наблюдений прогнозируете наиболее распространенный класс. Сравните метрики.

In [ ]:
y_t_naiv = target.value_counts(ascending=False).index[0]
y_t_pred = [y_t_naiv for i in range(len(target))]
print("Accuracy для наивного классификатора: ", accuracy_score(target, y_t_pred))

Accuracy для наивного классификатора:  0.6645745751672686


Accuracy для наивного классификатора хуже accuracy для RandomForestClassifier

* Снова проведите бинаризацию целевого столбца, преобразовав теперь целые числа в классы bad (0, 1) (невкусное), so-so (2, 3) (нормальное), great (4, 5) (вкусное).

In [ ]:
t_сlass = target.copy()
t_сlass = t_сlass.replace(0,'bad')
t_сlass = t_сlass.replace(1,'bad')
t_сlass = t_сlass.replace(2,'so-so')
t_сlass = t_сlass.replace(3,'so-so')
t_сlass = t_сlass.replace(4,'great')
t_сlass = t_сlass.replace(5,'great')
t_сlass

0        so-so
1        great
2        great
3        great
4        so-so
         ...  
20046    great
20047    so-so
20048    great
20050    great
20051    great
Name: rating, Length: 16889, dtype: object

In [ ]:
X_c_train, X_c_test, y_c_train, y_c_test = train_test_split(X, t_сlass, test_size=0.2, random_state=21, stratify=t_сlass)

* Снова для прогнозирования классов попробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе кроссвалидации и посчитайте accuracy на тестовой подвыборке.

Логистическая регрессия

In [ ]:
warnings.filterwarnings("ignore")
lr_cc_parametrs = { 'penalty':('l1', 'l2'),
                   'fit_intercept':(True,False),
                   'solver':('lbfgs', 'liblinear', 'saga')}
lr_cc_ = LogisticRegression(random_state=21)
lr_cc = GridSearchCV(lr_cc_, lr_cc_parametrs, scoring='accuracy', n_jobs=-1)
lr_cc.fit(X_c_train, y_c_train)
lr_cc.best_params_

{'fit_intercept': True, 'penalty': 'l1', 'solver': 'liblinear'}

In [ ]:
lr_cc_best=LogisticRegression(random_state=21, fit_intercept = True, penalty = 'l1', solver = 'liblinear')
print("accuracy для LogisticRegression: ", cross_val_score(lr_cc_best, X_c_train, y_c_train).mean())

accuracy для LogisticRegression:  0.7984602737370243


Дерево решений

In [ ]:
dtr_cc_params = {'criterion': ('entropy', 'gini'),
                'class_weight': ['balanced', None],
                'max_depth': range(1, 50)}
dtr_cc_ = DecisionTreeClassifier(random_state=21)
dtr_cc = GridSearchCV(dtr_cc_ , dtr_cc_params, scoring='accuracy', n_jobs=-1)
dtr_cc.fit(X_c_train, y_c_train)
dtr_cc.best_params_

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

In [ ]:
dtr_cc_best=DecisionTreeClassifier(random_state=21,class_weight= None, criterion= 'gini', max_depth= 2)
print("accuracy для DecisionTree: ", cross_val_score(dtr_cc_best, X_c_train, y_c_train).mean())

accuracy для DecisionTree:  0.7971281737839334


Случайный лес

In [ ]:
rfr_cc_params = {'max_depth': tuple(range(1, 50)),
                'n_estimators': (10, 50, 100),
                'criterion': ('entropy', 'gini'),
                'class_weight': ['balanced', None]}
rfc_cc_ = RandomForestClassifier(random_state=21)
rfc_cc = GridSearchCV(rfc_cc_ , rfr_cc_params,  scoring='accuracy', n_jobs=-1)
rfc_cc.fit(X_c_train, y_c_train)
rfc_cc.best_params_

{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 21,
 'n_estimators': 100}

In [ ]:
rfc_cc_best=RandomForestClassifier(random_state=21,class_weight=None,criterion='entropy',max_depth=21,n_estimators=100)
print("accuracy для RandomForestClassifier: ", cross_val_score(rfc_cc_best, X_c_train, y_c_train).mean())

accuracy для RandomForestClassifier:  0.7977204646645049


Лучшая модель Случайный лес

In [ ]:
rfc_cc_best=RandomForestClassifier(random_state=21,class_weight=None,criterion='entropy',max_depth=21,n_estimators=100)
print("accuracy для RandomForestClassifier: ", cross_val_score(rfc_cc_best, X_c_test, y_c_test).mean())

accuracy для RandomForestClassifier:  0.7975139162831469


Наивный классификатор

* Снова рассчитайте значение accuracy наивного классификатора, в котором вы для всех наблюдений прогнозируете наиболее распространенный класс. Сравните метрики.

In [ ]:
y_c_naiv = t_сlass.value_counts(ascending=False).index[0]
y_c_pred = [y_c_naiv for i in range(len(t_сlass))]
print("Accuracy для наивного классификатора: ", accuracy_score(t_сlass, y_c_pred))

Accuracy для наивного классификатора:  0.7941855645686542


Лучшая модель Случайный лес

* Что хуже: спрогнозировать плохой рейтинг, который на самом деле окажется хорошим, или спрогнозировать хороший рейтинг, который на самом деле окажется плохим? Замените метрику accuracy другой соответствующей метрикой.

In [ ]:
from sklearn.metrics import balanced_accuracy_score, f1_score, jaccard_score

In [ ]:
warnings.filterwarnings("ignore")
rfr_cc_best=RandomForestClassifier(random_state=21,class_weight=None,criterion='entropy',max_depth= 49,n_estimators=100).fit(X_c_train, y_c_train)
print("accuracy для RandomForest: ", accuracy_score(y_c_train, rfr_cc_best.predict(X_c_train)))
print("balanced_accuracy для RandomForest: ", balanced_accuracy_score(y_c_train, rfr_cc_best.predict(X_c_train)))
print("f1 для RandomForest: ", f1_score(y_c_train, rfr_cc_best.predict(X_c_train), average='weighted'))
print("precision для RandomForest: ", precision_score(y_c_train,rfr_cc_best.predict(X_c_train), average='weighted'))
print("recall для RandomForest: ", recall_score(y_c_train, rfr_cc_best.predict(X_c_train), average='weighted'))
print("jaccard для RandomForest: ", jaccard_score(y_c_train, rfr_cc_best.predict(X_c_train), average='weighted'))

accuracy для RandomForest:  0.8595218710680187
balanced_accuracy для RandomForest:  0.5517994068855239
f1 для RandomForest:  0.8284085513596218
precision для RandomForest:  0.8786611080252935
recall для RandomForest:  0.8595218710680187
jaccard для RandomForest:  0.7406349676683958


Лучшая метрика precision

* Для прогнозирования классов с новой метрикой опробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение и посчитайте метрику на тестовой подвыборке.

Логистическая регрессия

In [ ]:
warnings.filterwarnings("ignore")
lr_cc_parametrs = { 'penalty':( 'l2', None),
                   'fit_intercept':(True,False),
                   'solver':('lbfgs', 'liblinear', 'saga')}
lr_cc_ = LogisticRegression(random_state=21)
lr_cc = GridSearchCV(lr_cc_, lr_cc_parametrs, scoring='precision', n_jobs=-1)
lr_cc.fit(X_c_train, y_c_train)
lr_cc.best_params_

{'fit_intercept': True, 'penalty': 'l2', 'solver': 'lbfgs'}

In [ ]:
lr_cc_best=LogisticRegression(random_state=21, fit_intercept = True, penalty = 'l2', solver = 'lbfgs')
lr_cc_best= lr_cc_best.fit(X_c_train, y_c_train)
print("precision для LogisticRegression: ", precision_score(y_c_train, lr_cc_best.predict(X_c_train), average='weighted'))

precision для LogisticRegression:  0.766363899820862


Дерево решений

In [ ]:
dtr_cc_params = {'criterion': ('entropy', 'gini'),
                'class_weight': ['balanced', None],
                'max_depth': range(1, 50)}
dtr_cc_ = DecisionTreeClassifier(random_state=21)
dtr_cc = GridSearchCV(dtr_cc_ , dtr_cc_params, scoring='precision', n_jobs=-1)
dtr_cc.fit(X_c_train, y_c_train)
dtr_cc.best_params_

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 1}

In [ ]:
results_dtr =  pd.DataFrame(dtr_cc.cv_results_)
results_dtr = results_dtr[['param_max_depth', 'param_class_weight', "param_criterion",  "rank_test_score"]]
results_dtr.sort_values(by=['rank_test_score'], inplace=True)
results_dtr

,param_max_depth,param_class_weight,param_criterion,rank_test_score
0,1,balanced,entropy,1
125,28,None,entropy,2
126,29,None,entropy,3
127,30,None,entropy,4
128,31,None,entropy,5
...,...,...,...,...
68,20,balanced,gini,192
69,21,balanced,gini,193
70,22,balanced,gini,194
72,24,balanced,gini,195


In [ ]:
dtr_cc_best=DecisionTreeClassifier(random_state=21,max_depth=49, class_weight=None, criterion='gini')
dtr_cc_best=dtr_cc_best.fit(X_c_train, y_c_train)
print("precision для DecisionTree: ", precision_score(y_c_train, dtr_cc_best.predict(X_c_train), average='weighted'))

precision для DecisionTree:  0.893684830977541


Случайный лес

In [ ]:
rfr_cc_params = {'max_depth': tuple(range(49, 151)),
                'n_estimators': (50, 100),
                'criterion': ('entropy', 'gini'),
                'class_weight': ['balanced', None]}
rfc_cc_ = RandomForestClassifier(random_state=21)
rfc_cc = GridSearchCV(rfc_cc_ , rfr_cc_params,  scoring='precision', n_jobs=-1)
rfc_cc.fit(X_c_train, y_c_train)
rfc_cc.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 49,
 'n_estimators': 50}

In [ ]:
results_rfc =  pd.DataFrame(rfc_cc.cv_results_)
results_rfc = results_rfc[['param_max_depth', 'param_class_weight', "param_criterion",  "rank_test_score",'param_n_estimators']]
results_rfc.sort_values(by=['rank_test_score'], inplace=True)
results_rfc

,param_max_depth,param_class_weight,param_criterion,rank_test_score,param_n_estimators
0,49,balanced,entropy,1,50
538,114,None,entropy,2,50
539,114,None,entropy,3,100
540,115,None,entropy,4,50
541,115,None,entropy,5,100
...,...,...,...,...,...
275,84,balanced,gini,812,100
276,85,balanced,gini,813,50
277,85,balanced,gini,814,100
267,80,balanced,gini,815,100


In [ ]:
rfc_cc_best=RandomForestClassifier(random_state=21,class_weight=None,criterion='gini',max_depth=49,n_estimators=100)
rfc_cc_best=rfc_cc_best.fit(X_c_train, y_c_train)
print("precision для RandomForest: ", precision_score(y_c_train, rfc_cc_best.predict(X_c_train), average='weighted'))

precision для RandomForest:  0.8724362926170667


In [ ]:
rfc_cc_best=RandomForestClassifier(random_state=21,class_weight=None,criterion='gini',max_depth=150,n_estimators=100)
rfc_cc_best=rfc_cc_best.fit(X_c_train, y_c_train)
print("precision для RandomForest: ", precision_score(y_c_train, rfc_cc_best.predict(X_c_train), average='weighted'))

precision для RandomForest:  0.9712239425428628


Лучшая модель Случайный лес

In [ ]:
rfc_cc_best=RandomForestClassifier(random_state=21,class_weight=None,criterion='gini',max_depth= 150,n_estimators=100)
rfc_cc_best=rfc_cc_best.fit(X_c_train, y_c_train)
print("precision на test для RandomForest: ", precision_score(y_c_test, rfc_cc_best.predict(X_c_test), average='weighted'))

precision на test для RandomForest:  0.6868206662343674


* Попробуйте различные ансамбли и их гиперпараметры. Выберите лучшее решение и посчитайте метрику на тестовой подвыборке.

Bagging classifiers

In [ ]:
best_precision = 0

with tqdm(total=(3)) as pbar:
    for n_estimators in (25, 50, 100):
        
        bc1 = BaggingClassifier(base_estimator=
                                DecisionTreeClassifier(max_depth=49, class_weight=None, criterion='gini'),
                                   random_state=21, n_jobs=8, n_estimators=n_estimators)
        bc1.fit(X_c_train, y_c_train)
        pri = precision_score(y_c_train, bc1.predict(X_c_train), average='weighted')
        
        if pri > best_precision:
            best_precision = pri
            params = [n_estimators]
        pbar.update(1)
print ("Лучший n_estimators =",params[0])

  0%|          | 0/3 [00:00<?, ?it/s]

Лучший n_estimators = 25


In [ ]:
bc1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=49, class_weight=None, criterion='gini'),
                                       random_state=21, n_jobs=8, n_estimators=50)
bc1.fit(X_c_train, y_c_train)
print("precision для DecisionTree на Bagging classifiers", precision_score(y_c_train, bc1.predict(X_c_train), average='weighted'))

precision для DecisionTree на Bagging classifiers 0.878081861232459


In [ ]:
best_precision = 0

with tqdm(total=(3)) as pbar:
    for n_estimators in (25, 50, 100):
        
        bc2 = BaggingClassifier(base_estimator=
                                RandomForestClassifier(class_weight=None, criterion='gini',max_depth=150, n_estimators=100),
                                   random_state=21, n_estimators=n_estimators)
        bc2.fit(X_c_train, y_c_train)
        pri = precision_score(y_c_train, bc2.predict(X_c_train), average='weighted')
        
        if pri > best_precision:
            best_precision = pri
            params = [n_estimators]
        pbar.update(1)
print ("Лучший n_estimators =",params[0])

  0%|          | 0/3 [00:00<?, ?it/s]

Лучший n_estimators = 25


In [ ]:
bc2 = BaggingClassifier(base_estimator=RandomForestClassifier(class_weight=None,criterion='gini',max_depth=150,n_estimators=100),
                                       random_state=21,  n_estimators=25)
bc2.fit(X_c_train, y_c_train)
print("precision для RandomForest на Bagging classifiers", precision_score(y_c_train, bc2.predict(X_c_train), average='weighted'))

precision для RandomForest на Bagging classifiers 0.9065339154735824


Stacking classifiers

In [ ]:
warnings.filterwarnings('ignore')
print('При passthrough=True : \n')

for n in tqdm([2, 3, 4, 5, 6, 7]):
    print(n)
    k_fold = StratifiedKFold(n_splits=n, shuffle=True, random_state=21)
    stacking = StackingClassifier([('lr', lr_cc_best), ('tree', dtr_cc_best), ('forest', rfc_cc_best)
        ],passthrough=True, cv=k_fold)
    stacking.fit(X_c_train, y_c_train)
    
    print("precision для Stacking classifiers", precision_score(y_c_train, stacking.predict(X_c_train), average='weighted'))

print('------------------------------------------------------')
print('При passthrough=False :\n')

for n in tqdm([2, 3, 4, 5, 6, 7]):
    print(n)
    k_fold = StratifiedKFold(n_splits=n, shuffle=True, random_state=21)
    stacking = StackingClassifier([('lr', lr_cc_best),('tree', dtr_cc_best),('forest', rfc_cc_best)
        ],passthrough=False, cv=k_fold)
    stacking.fit(X_c_train, y_c_train)
    print("precision для Stacking classifiers", precision_score(y_c_train, stacking.predict(X_c_train), average='weighted'))

При passthrough=True : 



  0%|          | 0/6 [00:00<?, ?it/s]

2
precision для Stacking classifiers 0.7414323651840767
3
precision для Stacking classifiers 0.7433876420706674
4
precision для Stacking classifiers 0.735969249794025
5
precision для Stacking classifiers 0.7274064585099416
6
precision для Stacking classifiers 0.7307328748828986
7
precision для Stacking classifiers 0.7106249472531331
------------------------------------------------------
При passthrough=False :



  0%|          | 0/6 [00:00<?, ?it/s]

2
precision для Stacking classifiers 0.7295215298084624
3
precision для Stacking classifiers 0.7256523114535381
4
precision для Stacking classifiers 0.7237561531885985
5
precision для Stacking classifiers 0.8363550762878744
6
precision для Stacking classifiers 0.7215963873973221
7
precision для Stacking classifiers 0.7213971044282359


In [ ]:
k_fold = StratifiedKFold(n_splits=7, shuffle=True, random_state=21)
stacking = StackingClassifier([('lr', lr_cc_best), ('tree', dtr_cc_best), ('forest', rfc_cc_best)],
                              passthrough=True, 
                              cv=k_fold,
                              final_estimator=RandomForestClassifier(class_weight=None,criterion='gini',max_depth=150,n_estimators=100))
stacking.fit(X_c_train, y_c_train)
print("precision для Stacking classifiers", precision_score(y_c_train, stacking.predict(X_c_train), average='weighted'))

precision для Stacking classifiers 0.7903523421396903


Лучший ансамбль RandomForest на Bagging classifiers

In [ ]:
bc_best = BaggingClassifier(base_estimator=RandomForestClassifier(class_weight=None,criterion='gini',max_depth=150,n_estimators=100),
                                       random_state=21,  n_estimators=25)
bc_best.fit(X_c_train, y_c_train)
print("precision для RandomForest на Bagging classifiers", precision_score(y_c_test, bc_best.predict(X_c_test), average='weighted'))

precision для RandomForest на Bagging classifiers 0.7066597665268143


### 4. Принятие решения

* Примите решение о том, что лучше использовать: регрессионную модель или классификацию. Сохраните выбранную модель. Вы будете использовать ее в основной программе.

Лучшая модель Случайный лес

In [ ]:
rfc_cc_best=RandomForestClassifier(random_state=21, class_weight=None, criterion='gini', max_depth=49, n_estimators=50)
rfc_cc_best=rfc_cc_best.fit(X_c_train, y_c_train)

In [ ]:
joblib.dump(rfc_cc_best, 'best_RandomForestClassifier.joblib')

['best_RandomForestClassifier.joblib']

## 2. Пищевая ценность

1. Соберите в датафрейм всю информацию о пищевой ценности продуктов из подготовленного вами и отфильтрованного набора данных (только столбцы с продуктами). Используйте для этого следующий API.

In [8]:
def search_for_nutritional_value(food_product):
    food_product = food_product
    url = f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key=6bDiul26iNimzsMAN6nUbUrfXthZwrImPBJQZWCO&query={food_product}'
    requests.headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 YaBrowser/21.8.3.607 Yowser/2.5 Safari/537.36'}
    res = requests.get(url).json()
    Nutrients ={}
    try:
        x = res['foods'][0]['foodNutrients']
        for i in x:
            Nutrients[i['nutrientName']] = [i['value'], i['unitName']]
        return Nutrients
    except IndexError:
        Nutrients =0
        return Nutrients

In [9]:
def conversion_to_table(Nutrients,food_product):
    cc = pd.DataFrame(list(pp.items()),columns=['nutrition', 'value'])
    cc['measure'] = 0
    cc['food_product'] = food_product
    for i in range(len(cc)):
        x = cc.iloc[[i],[1]].value
        y = x[i]
        a = y[0]
        b = y[1]
        cc.iloc[[i],[1]] = a
        cc.iloc[[i],[2]] = b
        cc.iloc[[i],[3]] = food_product
    return cc

In [ ]:
x = 'almond' #проверка
pp = search_for_nutritional_value(x)
cc = conversion_to_table(pp,x)
df_new = cc
df_new

,nutrition,value,measure,food_product
0,Protein,21.0,G,almond
1,Total lipid (fat),55.5,G,almond
2,"Carbohydrate, by difference",18.8,G,almond
3,Energy,614,KCAL,almond
4,"Alcohol, ethyl",0.0,G,almond
...,...,...,...,...
60,PUFA 20:5 n-3 (EPA),0.0,G,almond
61,MUFA 22:1,0.0,G,almond
62,PUFA 22:5 n-3 (DPA),0.0,G,almond
63,"Fatty acids, total monounsaturated",32.4,G,almond


In [ ]:
df_ingr = X.copy()
df_ingr

,alcoholic,almond,amaretto,anchovy,anise,apple,apple juice,apricot,artichoke,arugula,...,watermelon,wheat/gluten-free,whiskey,white wine,wild rice,wine,yogurt,yuca,zucchini,turkey
0,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,5.223962,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,7.094670
1,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
2,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
3,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
4,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20046,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
20047,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
20048,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,10.708785,-0.125041,...,-0.064974,1.719090,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951
20050,-0.21497,-0.181392,-0.046218,-0.050523,-0.078333,-0.191426,-0.018852,-0.12057,-0.093381,-0.125041,...,-0.064974,-0.581703,-0.055035,-0.165451,-0.031743,-0.101136,-0.171948,-0.018852,-0.131014,-0.140951


In [ ]:
df_ingr.info(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16889 entries, 0 to 20051
Data columns (total 311 columns):
 #    Column                Dtype  
---   ------                -----  
 0    alcoholic             float64
 1    almond                float64
 2    amaretto              float64
 3    anchovy               float64
 4    anise                 float64
 5    apple                 float64
 6    apple juice           float64
 7    apricot               float64
 8    artichoke             float64
 9    arugula               float64
 10   asian pear            float64
 11   asparagus             float64
 12   avocado               float64
 13   bacon                 float64
 14   banana                float64
 15   barley                float64
 16   bean                  float64
 17   beef                  float64
 18   beef rib              float64
 19   beef shank            float64
 20   beef tenderloin       float64
 21   beer                  float64
 22   beet                

In [ ]:
#df_ingr1 = pd.DataFrame(df_ingr.columns.tolist(), columns=['title']).drop(labels = [28,45,80,125,132,140,174,211,278,280,310],axis = 0)
df_ingr1 = pd.DataFrame(df_ingr.columns.tolist(), columns=['title']).drop(labels = [47,58,85,86,87,188,269,277,302],axis = 0)
df_ingr1.reset_index(drop=True, inplace=True)
df_ingr1

,title
0,alcoholic
1,almond
2,amaretto
3,anchovy
4,anise
...,...
297,wine
298,yogurt
299,yuca
300,zucchini


In [ ]:
for i in range(len(df_ingr1)):
    x = df_ingr1['title'].values[i]
    if i == 0 :
        pp = search_for_nutritional_value(x)
        cc = conversion_to_table(pp,x)
        df_new = cc
    if i > 0:
        pp = search_for_nutritional_value(x)
        if pp != 0:
            cc = conversion_to_table(pp,x)
            df_new = pd.concat([df_new, cc])

In [ ]:
df_new

,nutrition,value,measure,food_product
0,Galactose,0.0,G,alcoholic
1,"Fiber, total dietary",0.0,G,alcoholic
2,"Iron, Fe",0.03,MG,alcoholic
3,"Magnesium, Mg",12.0,MG,alcoholic
4,"Phosphorus, P",32.0,MG,alcoholic
...,...,...,...,...
9,"Vitamin A, IU",530,IU,turkey
10,"Vitamin C, total ascorbic acid",0.0,MG,turkey
11,Cholesterol,34.0,MG,turkey
12,"Fatty acids, total trans",0.0,G,turkey


In [ ]:
df_new.to_csv('df_new.csv', index = False)

2. Конвертируйте все значения в % от суточной нормы потребления. Сохраняйте только те ингридиенты, которые находятся в этой и этой таблицах.

In [ ]:
"""
df_vit = pd.DataFrame(pd.read_excel('compare.xlsx'))
df_vit=df_vit.dropna()
df_vit=df_vit.reset_index(drop=True)
df_vit
"""

In [10]:
tabula.convert_into('Daily-Reference-Values-_DRVs_-under-the-New-NFL.pdf', 
                    output_format="csv", pages='all', 
                    output_path='Daily-Reference-Values-_DRVs_-under-the-New-NFL.csv')

Dec 28, 2022 3:46:53 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Dec 28, 2022 3:46:53 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>



In [12]:
tabula.convert_into('Reference-Daily-Intakes-_RDIs_-in-the-New-Nutrition-Facts-Label.pdf', 
                    output_format="csv", pages='all', 
                    output_path='Reference-Daily-Intakes-_RDIs_-in-the-New-Nutrition-Facts-Label.csv')
Reference_Daily = pd.read_csv('Reference-Daily-Intakes-_RDIs_-in-the-New-Nutrition-Facts-Label.csv')
Reference_Daily.drop(Reference_Daily.tail(3).index,inplace=True)
Reference_Daily.drop(Reference_Daily.head(1).index,inplace=True)
Reference_Daily.to_csv('Reference-Daily-Intakes-_RDIs_-in-the-New-Nutrition-Facts-Label.csv')

In [18]:
Daily1 = pd.read_csv('Daily-Reference-Values-_DRVs_-under-the-New-NFL.csv')
Daily1.columns = Daily1.iloc[0]
Daily1.drop (index=Daily1.index [0], axis= 0 , inplace= True )
Daily1

,Food Component,Unit of measure,Adults and Children ≥ 4\nyears,Infants through 12\nmonths,Children 1 through 3\nyears,Pregnant women and lactating women
1,Fat,Grams (g),78,30,39,78
2,Saturated fat,Grams (g),20,NaN,10,20
3,Cholesterol,Milligrams (mg),300,NaN,300,300
4,Total\ncarbohydrates,Grams (g),275,95,150,275
5,Sodium,Milligrams (mg),2300,NaN,1500,2300
6,Dietary Fiber,Grams (g),28,NaN,14,28
7,Protein,Grams (g),50,NaN,13,NaN
8,Added sugars,Grams (g),50,NaN,25,50


In [27]:
Daily2 = pd.read_csv('Reference-Daily-Intakes-_RDIs_-in-the-New-Nutrition-Facts-Label.csv')
#del Daily2['Unnamed: 0']
Daily2.columns = Daily1.columns 
#Daily2.columns = Daily2.iloc[0]
Daily2.drop (index=Daily2.index [0], axis= 0 , inplace= True )
Daily2

,Food Component,Unit of measure,Adults and Children ≥ 4\nyears,Infants through 12\nmonths,Children 1 through 3\nyears,Pregnant women and lactating women
1,Vitamin A,Micrograms RAE (mcg),900,500,300,1300
2,Vitamin C,Milligrams (mg),90,50,15,120
3,Calcium,Milligrams (mg),1300,260,700,1300
4,Iron,Milligrams (mg),18,11,7,27
5,Vitamin D,Micrograms (mcg),20,10,15,15
6,Vitamin E,Milligrams (mg)4,15,5,6,19
7,Vitamin K,Micrograms (mcg),120,2.5,30,90
8,Thiamin,Milligrams (mg),1.2,0.3,0.5,1.4
9,Riboflavin,Milligrams (mg),1.3,0.4,0.5,1.6
10,Niacin,Milligrams NE (mg),16,4,6,18


In [28]:
Daily = pd.concat([Daily1, Daily2])
Daily =Daily.reset_index(drop=True)
Daily

,Food Component,Unit of measure,Adults and Children ≥ 4\nyears,Infants through 12\nmonths,Children 1 through 3\nyears,Pregnant women and lactating women
0,Fat,Grams (g),78,30,39,78
1,Saturated fat,Grams (g),20,NaN,10,20
2,Cholesterol,Milligrams (mg),300,NaN,300,300
3,Total\ncarbohydrates,Grams (g),275,95,150,275
4,Sodium,Milligrams (mg),2300,NaN,1500,2300
5,Dietary Fiber,Grams (g),28,NaN,14,28
6,Protein,Grams (g),50,NaN,13,NaN
7,Added sugars,Grams (g),50,NaN,25,50
8,Vitamin A,Micrograms RAE (mcg),900,500,300,1300
9,Vitamin C,Milligrams (mg),90,50,15,120


In [143]:
vit = pd.ExcelFile("Daily_Values_DRVs_and_RDIs.xlsx").parse("Лист2")
#vit = pd.DataFrame(pd.read_excel('Daily_Values_DRVs_and_RDIs.xlsx')).parse("Лист2")
#vit['norm'] = Daily['Adults and Children ≥ 4\nyears']
for i in range(len(vit)):
  try:
    vit.loc[i,'norm'] = Daily.loc[Daily['Food Component'] == vit.iloc[i,0]]['Adults and Children ≥ 4\nyears'].values[0]
    vit.loc[i,'units'] = Daily.loc[Daily['Food Component'] == vit.iloc[i,0]]['Unit of measure'].values[0]
  except IndexError:
    continue

vit = vit.dropna()
vit = vit.reset_index(drop=True)
vit.to_csv('Daily_Values_DRVs_and_RDIs.csv',index = False)
vit

,Food\rComponent,Component2,norm,units
0,Vitamin A,"Vitamin A, RAE",900,Micrograms RAE (mcg)
1,Vitamin C,"Vitamin C, total ascorbic acid",90,Milligrams (mg)
2,Calcium,"Calcium, Ca",1300,Milligrams (mg)
3,Iron,"Iron, Fe",18,Milligrams (mg)
4,Vitamin D,Vitamin D (D2 + D3),20,Micrograms (mcg)
5,Vitamin E,Vitamin E (alpha-tocopherol),15,Milligrams (mg)4
6,Thiamin,Thiamin,1.2,Milligrams (mg)
7,Riboflavin,Riboflavin,1.3,Milligrams (mg)
8,Niacin,Niacin,16,Milligrams NE (mg)
9,Vitamin B6,Vitamin B-6,1.7,Milligrams (mg)


In [144]:
df_new = pd.read_csv('df_new.csv')

In [145]:
df_new

,nutrition,value,measure,food_product
0,Galactose,0.00,G,alcoholic
1,"Fiber, total dietary",0.00,G,alcoholic
2,"Iron, Fe",0.03,MG,alcoholic
3,"Magnesium, Mg",12.00,MG,alcoholic
4,"Phosphorus, P",32.00,MG,alcoholic
...,...,...,...,...
11132,"Vitamin A, IU",530.00,IU,turkey
11133,"Vitamin C, total ascorbic acid",0.00,MG,turkey
11134,Cholesterol,34.00,MG,turkey
11135,"Fatty acids, total trans",0.00,G,turkey


In [146]:
from logging import exception
a = vit['Component2']
for i in range(len(df_new)):
  try:  
    x = df_new['nutrition'].values[i]
    if x not in a:
        df_new.drop(index=df_new.index[i], axis= 0 , inplace= True )
  except IndexError:
    break

3. Сохраните измененный датафрейм в CSV-файл, который вы будете использовать в своей основной программе.

In [147]:
df_new.to_csv('Nutrition.csv',index = False)
df_new = pd.read_csv('Nutrition.csv')

In [148]:
df_new

,nutrition,value,measure,food_product
0,"Fiber, total dietary",0.00,G,alcoholic
1,"Magnesium, Mg",12.00,MG,alcoholic
2,"Sodium, Na",4.00,MG,alcoholic
3,Ash,0.12,G,alcoholic
4,Lactose,0.00,G,alcoholic
...,...,...,...,...
5563,"Sugars, total including NLEA",1.69,G,turkey
5564,"Calcium, Ca",42.00,MG,turkey
5565,"Sodium, Na",521.00,MG,turkey
5566,"Vitamin C, total ascorbic acid",0.00,MG,turkey


## 3. Похожие рецепты

1. Для каждого рецепта из набора данных найдите ссылку на сайте epicurious.com и подробную информацию о нем (название рецепта, рейтинг на платформе и URl). Если сделать это не удалось, найдите для данного рецепта похожую ссылку в Интернете.


In [149]:
def pre_processing(title):    
    title = title.rstrip() 
    title = unidecode.unidecode(title)
    title = title.replace('"','')
    title = title.replace('’','')
    title = title.replace(' ', '%20')
    return title

In [150]:
def search_url(recipe): 
    url = f'http://www.epicurious.com/search/{recipe}?content=recipe'
    HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Firefox/3.0.7 Chrome/104.0.5112.102'}
    b12 = BeautifulSoup(request.urlopen(request.Request(url,headers=HEADERS)).read(),"html.parser")
    try:
        recipe_url_ = b12.select('div.results-group article.recipe-content-card a.view-complete-item') 
        recipe_url = [i['href'] for i in recipe_url_ ]
        return [(url + i) for i in recipe_url ]
    except (HTTPError, URLError):
        return []

In [180]:
def search_rating(recipe): 
    url = f'http://www.epicurious.com/search/{recipe}?content=recipe'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Firefox/3.0.7 Chrome/104.0.5112.102'}
    b12 = BeautifulSoup(request.urlopen(request.Request(url,headers)).read(),"html.parser")
    try:
        recipe_rating_ = b12.select('div.results-group article.recipe-content-card dl.recipes-ratings-summary') 
        recipe_rating = [i['data-reviews-rating'] for i in recipe_rating_ ]
        return recipe_rating
    except (HTTPError, URLError):
        return []

In [161]:
df = pd.read_csv('epi_r_new.csv')
df

,title,rating,alcoholic,almond,amaretto,anchovy,anise,apple,apple juice,apricot,...,watermelon,wheat/gluten-free,whiskey,white wine,wild rice,wine,yogurt,yuca,zucchini,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [162]:
df_recipes = df[['title','rating','almond']]
df_recipes =df_recipes.rename(columns = {'almond' : 'url'})
df_recipes

,title,rating,url
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,0.0
2,Potato and Fennel Soup Hodge,3.750,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,0.0
4,Spinach Noodle Casserole,3.125,0.0
...,...,...,...
20047,Parmesan Puffs,3.125,0.0
20048,Artichoke and Parmesan Risotto,4.375,0.0
20049,Turkey Cream Puff Pie,4.375,0.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,0.0


In [187]:
warnings.filterwarnings("ignore")
for i in range(#len(df)):
    x = df['title'].values[i]
    x = pre_processing(x)
    df_recipes['rating'][i] = search_rating(x)
    df_recipes['url'][i] = search_url(x)
    df_recipes.to_csv('recipes.csv',index = False)

2. Сохраните новый датафрейм в CSV-файл, который вы будете использовать в своей основной программе.

In [189]:
recipes = pd.read_csv('recipes.csv')
recipes

,title,url,class,val,meal,rating,almond,amaretto,anchovy,anise,...,watermelon,whiskey,white wine,whole wheat,wild rice,wine,yogurt,yuca,zucchini,turkey
0,"Lentil, Apple, and Turkey Wrap","http://www.epicurious.com/search/Lentil,%20App...",so-so,8,unknown,2.500,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,http://www.epicurious.com/search/Boudin%20Blan...,great,4,unknown,4.375,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,http://www.epicurious.com/search/Potato%20and%...,great,4,lunch,3.750,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,http://www.epicurious.com/search/Mahi-Mahi%20i...,great,2,unknown,5.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,http://www.epicurious.com/search/Spinach%20Noo...,so-so,3,unknown,3.125,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19907,Parmesan Puffs,http://www.epicurious.com/search/Parmesan%20Pu...,so-so,3,unknown,3.125,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19908,Artichoke and Parmesan Risotto,http://www.epicurious.com/recipes/food/views/a...,great,2,dinner,4.375,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19909,Turkey Cream Puff Pie,http://www.epicurious.com/recipes/food/views/t...,great,5,dinner,4.375,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
19910,Snapper on Angel Hair with Citrus Cream,http://www.epicurious.com/recipes/food/views/s...,great,7,unknown,4.375,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Основная программа
1. Она принимает на вход список ингредиентов.
2. Она создает прогноз и выдает рейтинг с оценкой (невкусное, нормальное, вкусное) блюда, которое можно приготовить на основе списка переданных ингредиентов.
3. Она находит и выводит информацию обо всех питательных веществах (белках, жирах, натрии и т. д.) продуктов из списка, а также их объем в процентах от суточной нормы потребления.
4. Она отображает три рецепта блюд, в которых используется максимальное количество продуктов из списка, с указанием их оценки и ссылок, по которым пользователь сможет найти полную информацию.

Пример работы программы:

    $ ./nutritionist.py milk, honey, jam
    I. НАШ ПРОГНОЗ

    Невкусное. Хоть конкретно вам может быть и понравится блюдо из этих ингредиентов, но, на наш взгляд, это плохая идея – готовить блюдо из них. Хотели предупредить.

    II. ПИЩЕВАЯ ЦЕННОСТЬ
    Milk
    Protein – 6% of Daily Value
    Total Carbohydrate – 1% of Daily Value
    Total Fat – 1% of Daily Value
    Calcium – 12% of Daily Value
    …

    Honey
    …

    Jam
    …

    III. ТОП-3 ПОХОЖИХ РЕЦЕПТА:
    - Strawberry-Soy Milk Shake, рейтинг: 3.0, URL:https://www.epicurious.com/recipes/food/views/strawberry-soy-milk-shake-239217
    - …
    - …

# Бонусная часть
Добавьте в классы дополнительные методы, которые помогут скрипту выполнить новую функцию: составление меню на день.

В ежедневном меню случайным образом должен выводиться список из трех рецептов, которые покрывают большую часть потребностей в питании (% от суточной пищевой ценности) и имеют самую высокую общую оценку.
Вы должны предлагать только рецепты, которые подходят для завтрака, обеда и ужина, соответственно.